Parse OSM file and export the result in CSV

In [45]:
import xml.etree.ElementTree
import csv
import re

In [46]:
dataFileName = 'daensen.osm'

In [ ]:
files = {'node':'nodes.csv', 'way':'ways.csv', 'nodes_tags': 'nodes_tags.csv'}
    
fields = {'node': ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp'],
           'way': ['id', 'user', 'uid', 'version', 'changeset', 'timestamp'],
           'nodes_tags': ['id', 'key', 'value', 'type']}

def initCSVfiles():
    fileDescriptor = {}
    for f in files:
        print('init {}'.format(files[f]))
        fileId = open(files[f], 'w')
        csvWriter = csv.writer(fileId, delimiter=';',
                            quotechar='\'', quoting=csv.QUOTE_MINIMAL)
        csvWriter.writerow(fields[f])
        fileDescriptor[f] = {}
        fileDescriptor[f]['fileId'] = fileId
        fileDescriptor[f]['csvWriter'] = csvWriter
        
    return fileDescriptor

def closeCSVfiles(fileDesc):
    for f in fileDesc:
        print(

In [93]:
files = {'node':'nodes.csv', 'way':'ways.csv', 'nodes_tags': 'nodes_tags.csv'}
    
fields = {'node': ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp'],
           'way': ['id', 'user', 'uid', 'version', 'changeset', 'timestamp'],
           'nodes_tags': ['id', 'key', 'value', 'type']}

def initCSVfiles():
    fileDescriptor = {}
    for f in files:
        #print('init {}'.format(files[f]))
        fileId = open(files[f], 'w')
        csvWriter = csv.writer(fileId, delimiter=';',
                            quotechar='\'', quoting=csv.QUOTE_MINIMAL)
        csvWriter.writerow(fields[f])
        fileDescriptor[f] = {}
        fileDescriptor[f]['fileId'] = fileId
        fileDescriptor[f]['csvWriter'] = csvWriter
        
    return fileDescriptor

def closeCSVfiles(fileDesc):
    for f in fileDesc:
        #print(fileDesc[f]['fileId'])
        fileDesc[f]['fileId'].close()

def writeElem(elemType, elem, fileDesc):
    record = []
    for i in fields[elemType]:
        record.append(elem[i])
    
    fileDesc[elemType]['csvWriter'].writerow(record)
    #print('Write in {}: {}'.format(fileDesc[elemType], record))
    
def parseElement(element, fileDesc):
    targetElement = ['node', 'way']
    nodeAttribs =  ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
    wayAttribs = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
    
    nodes = []
    ways = []
    
    if element.tag in targetElement:
        #print('Parsing: {}'.format(element.tag))
        selectedAttribs = fields[element.tag]

        attribs = element.attrib
        elem = {}
        for i in selectedAttribs:
            elem[i] = attribs[i]
            
        writeElem(element.tag, elem, fileDesc)
        #print(elem)
        
def parseOSMfile(fileName):
    fileDesc = initCSVfiles()
    #print(fileDesc)
    e = xml.etree.ElementTree.parse(fileName).getroot()
    for element in e:
        parseElement(element, fileDesc)
    closeCSVfiles(fileDesc)

In [94]:
parseOSMfile(dataFileName)